In [158]:
import requests
import pandas as pd
import numpy as np
import lxml.html as lh

The url to the wikipage from where the data has to be picked is saved in the variable website_url
The contents of the wikipage is fetched and based on the tag <tr> on the html page and the column count which is 3 the html is parsed to create a dataframe

In [159]:
website_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = requests.get(website_url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')    

In [160]:
tr_elements = doc.xpath('//tr')

col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [161]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [200]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

#The column headers are renamed
df.rename(columns={'Postcode':'PostalCode','Neighbourhood\n':'Neighborhood'}, inplace=True)
df['Neighborhood'] = df['Neighborhood'].map(lambda x: x.rstrip('\n'))

#rows having Borough column value as Not assigned is dropped
indexNames = df[ df['Borough'] == 'Not assigned' ].index
df.drop(indexNames , inplace=True)

#rows having Neighbourhood column value as Not assigned is replaced by Borough column value
df['Neighborhood'] = np.where(df.Neighborhood!='Not assigned', df.Neighborhood, df.Borough)

df.shape

(212, 3)

In [201]:
#where multiple rows exist for a postal code the rows are combined and the values in the Neighborhood column in concatenated using a ,
df.set_index(['PostalCode', 'Borough'], inplace=True)
result = df.groupby(level=['PostalCode','Borough'], sort=False).agg( ','.join)
print(result)

                                                                  Neighborhood
PostalCode Borough                                                            
M3A        North York                                                Parkwoods
M4A        North York                                         Victoria Village
M5A        Downtown Toronto                           Harbourfront,Regent Park
M6A        North York                          Lawrence Heights,Lawrence Manor
M7A        Queen's Park                                           Queen's Park
M9A        Etobicoke                                          Islington Avenue
M1B        Scarborough                                           Rouge,Malvern
M3B        North York                                          Don Mills North
M4B        East York                            Woodbine Gardens,Parkview Hill
M5B        Downtown Toronto                            Ryerson,Garden District
M6B        North York                               

In [171]:
#there are 103 unique postal code values in the dataframe
result.shape

(103, 1)

Read the csv file into a dataframe

In [183]:
Geospatial_df = pd.read_csv('https://cocl.us/Geospatial_data')
Geospatial_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
Geospatial_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [205]:
#Reset the multi index
result.reset_index(level=[0,1])

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [206]:
Geospatial_df

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [207]:
#Join the two dataframes based on the key PostalCode

result.join(Geospatial_df.set_index('PostalCode'), on='PostalCode')

,,Neighborhood,Latitude,Longitude
PostalCode,Borough,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M3B,North York,Don Mills North,43.745906,-79.352188
M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
